# CanDev2022 Challenge: Whales and ships: shall they never meet
### Team Name: The Killer Whales

This code converts the provided AIS data from CSV files into a raster format that is much smaller (from 490 MB down to 3 MB) and more suitable for quick interactions in our R Shiny app.

Location precision is relatively low in the produced raster files; this was chosen to match the other provided raster dataset on whale habitat suitability. Keeping both datasets on the same scale facilitates direct comparisons within the app.

This code also begins work to bring data into a single table based on the AIS CSV's, to facilitate future experiments with machine learning algorithms for model generation and optimization.

Code was run on Google Colaboratory, with the following folder structure in Google Drive:

/CanDev22

. ../data

. ../ais .........Provided AIS data (CSV files)

. ../dsz ........Designated Slow Zones (geojson)

. ../tif ...........Whale habitat files (tif)

. . ../new .....Converted AIS data (tif)

In [1]:
!python3 -m pip install rasterio
import rasterio
import pandas as pd
import numpy as np
import os
import json
from shapely.geometry import Point
from shapely.geometry import Polygon

     |████████████████████████████████| 19.3 MB 1.2 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Add a 'SpeedViolation' boolean attribute to each record

In [3]:
# Load all monthly files of the AIS data (csv files) into one dataframe.
# Many columns are currently irrelevant, but are kept as attributes for
# future work on model generation / optimization with machine learning.

ais_dir = "/content/gdrive/MyDrive/CanDev22/data/ais"
df_ais  = pd.DataFrame()

for ais_file in os.listdir(ais_dir):
  if ais_file.endswith(".csv"):
    ais_filepath = os.path.join(ais_dir, ais_file)
    #print(ais_filepath)
    df_ais = df_ais.append(pd.read_csv(ais_filepath))

print('Number of records: %d' % df_ais.shape[0])
print('Number of columns: %d' % df_ais.shape[1])
print('\nColumn names: %s' % df_ais.columns)
#df_ais.describe(include='all')

Number of records: 5016275
Number of columns: 15

Column names: Index(['MMSI', 'UTC_Timestamp', 'Lat', 'Lon', 'SOG', 'COG', 'Heading',
       'VesselName', 'VesselType', 'Status', 'Length', 'Width', 'Draft',
       'Cargo', 'TransceiverClass'],
      dtype='object')


In [4]:
# Load the Designated Slow Zones (geojson files)

dsz_dir = "/content/gdrive/MyDrive/CanDev22/data/dsz/"

with open(dsz_dir+'Block_Island_Sound_Nov_1-Apr_30.geojson', 'r') as json_file:
       zone1_json = json.load(json_file)
with open(dsz_dir+'Cape_Cod_Bay_Jan_1-May_15.geojson', 'r') as json_file:
       zone2_json = json.load(json_file)
with open(dsz_dir+'Great_South_Channel_Apr_1-Jul_31.geojson', 'r') as json_file:
       zone3_json = json.load(json_file)
with open(dsz_dir+'Off_Race_Point_Mar_1-Apr_30.geojson', 'r') as json_file:
       zone4_json = json.load(json_file)

# Set up zone 'seasons' when in effect
# (Zone 1 has two date ranges in this single year of data, as it spans Dec/Jan)

z1a_start = pd.Timestamp(2020,1,1)
z1a_end   = pd.Timestamp(2020,4,30)
z1b_start = pd.Timestamp(2020,11,1)
z1b_end   = pd.Timestamp(2020,12,31)
z2_start  = pd.Timestamp(2020,1,1)
z2_end    = pd.Timestamp(2020,5,15)
z3_start  = pd.Timestamp(2020,4,1)
z3_end    = pd.Timestamp(2020,7,31)
z4_start  = pd.Timestamp(2020,3,1)
z4_end    = pd.Timestamp(2020,4,30)

# Create zone boundary Polygons and confirm they're as expected with test points

zone1_coords = zone1_json['features'][0]['geometry']['coordinates'][0]
zone1_bounds = Polygon(zone1_coords)
zone2_coords = zone2_json['features'][0]['geometry']['coordinates'][0]
zone2_bounds = Polygon(zone2_coords)
zone3_coords = zone3_json['features'][0]['geometry']['coordinates'][0]
zone3_bounds = Polygon(zone3_coords)
zone4_coords = zone4_json['features'][0]['geometry']['coordinates'][0]
zone4_bounds = Polygon(zone4_coords)

#one point inside each zone
point1 = Point(-71.343, 40.959)
point2 = Point(-70.3832,41.9971)
point3 = Point(-68.4865,42.0559)
point4 = Point(-70.0045,42.1995)

#check zones each with one true and one false example 
print('Zone1: ', zone1_bounds.contains(point1), zone1_bounds.contains(point2))
print('Zone2: ', zone2_bounds.contains(point2), zone2_bounds.contains(point3))
print('Zone3: ', zone3_bounds.contains(point3), zone3_bounds.contains(point4))
print('Zone4: ', zone4_bounds.contains(point4), zone4_bounds.contains(point1))

Zone1:  True False
Zone2:  True False
Zone3:  True False
Zone4:  True False


In [5]:
# Create 'time' (pd.Timestamp datatype) from "UTC_Timestamp" for easy comparisons
df_ais['time'] = df_ais.apply(lambda row: pd.Timestamp(row.UTC_Timestamp), axis=1)

In [6]:
# Create 'SpeedViolation' (boolean) from ship location, date, and speed.

def speeding(rows):
  return ((rows.SOG > 10) and (rows.Length >= 19.8) and (
                #check if in zone1 during dates
                (((rows.time >= z1a_start and 
                   rows.time <= z1a_end) or
                  (rows.time >= z1b_start and
                   rows.time <= z1b_end)) and
                 (zone1_bounds.contains(Point(rows.Lon, rows.Lat)))) or
                 #check if in zone2 during dates
                 ((rows.time >= z2_start and 
                   rows.time <= z2_end) and
                 (zone2_bounds.contains(Point(rows.Lon, rows.Lat)))) or
                 #check if in zone3 during dates
                 ((rows.time >= z3_start and 
                   rows.time <= z3_end) and
                 #check if in zone4 during dates
                 (zone3_bounds.contains(Point(rows.Lon, rows.Lat)))) or
                 ((rows.time >= z4_start and 
                   rows.time <= z4_end) and
                 (zone4_bounds.contains(Point(rows.Lon, rows.Lat)))) )
  )

df_ais['SpeedViolation'] = df_ais.apply(speeding, axis = 'columns')

In [7]:
# Check new structure of dataframe
print('Number of records: %d' % df_ais.shape[0])
print('Number of columns: %d' % df_ais.shape[1])
print('\nColumn names: %s\n' % df_ais.columns)
#df_ais.describe(include='all')
df_ais.SpeedViolation.value_counts()

Number of records: 5016275
Number of columns: 17

Column names: Index(['MMSI', 'UTC_Timestamp', 'Lat', 'Lon', 'SOG', 'COG', 'Heading',
       'VesselName', 'VesselType', 'Status', 'Length', 'Width', 'Draft',
       'Cargo', 'TransceiverClass', 'time', 'SpeedViolation'],
      dtype='object')



False    5013644
True        2631
Name: SpeedViolation, dtype: int64

## Generate raster AIS data, for streamlined use with R Shiny app.


In [8]:
# Load some habitat suitability raster data;
#  just to copy the formatting, the R Shiny app will use the habitat raster data

tif_dir = "/content/gdrive/MyDrive/CanDev22/data/tif/"

raster_data = rasterio.open(tif_dir+'NARW_Apr_Norm.tif')
raster = raster_data.read(1)

# Check some meta-data, and a test point value 
raster_inds = raster_data.index(-69.4668,41.2763)
raster_val = raster[raster_inds[0], raster_inds[1]]
print('Old raster CRS: %s' % raster_data.crs)
print('Old raster bounds: %s' % str(raster_data.bounds))
print('Old raster grid data type: %s' % type(raster))
print('Old raster grid dimensions: %s' % str(raster.shape))
print('Old raster grid indices: %s' % str(raster_inds))
print('Old raster grid cell value: %.3f' % raster_val)

# Test a method for generating a new raster data file
new_data = rasterio.open(tif_dir+'new.tif', mode='w+', driver=raster_data.driver,
                    width=256, height=262, count=1, dtype=raster.dtype,
                    crs=raster_data.crs, transform=raster_data.transform)
new = new_data.read(1)

# Confirm the meta-data, and a test point value are as expected
new_inds = new_data.index(-69.4668,41.2763)
new_val = new[new_inds[0], new_inds[1]]
print('New raster CRS: %s' % new_data.crs)
print('New raster bounds: %s' % str(new_data.bounds))
print('New raster grid data type: %s' % type(new))
print('New raster grid dimensions: %s' % str(new.shape))
print('New raster grid indices: %s' % str(new_inds))
print('New raster grid cell value: %.3f' % new_val)

Old raster CRS: EPSG:4326
Old raster bounds: BoundingBox(left=-82.6965062291361, bottom=22.9352776326434, right=-53.427044387514854, top=52.89074248617764)
Old raster grid data type: <class 'numpy.ndarray'>
Old raster grid dimensions: (262, 256)
Old raster grid indices: (101, 115)
Old raster grid cell value: 0.191
New raster CRS: EPSG:4326
New raster bounds: BoundingBox(left=-82.6965062291361, bottom=22.9352776326434, right=-53.427044387514854, top=52.89074248617764)
New raster grid data type: <class 'numpy.ndarray'>
New raster grid dimensions: (262, 256)
New raster grid indices: (101, 115)
New raster grid cell value: 0.000


In [9]:
# Raster AIS data will factor in this lethality model based on ship speed.

data_points = {0: 0,
               3: 0.025813692,
               5: 0.053872054,
               7: 0.119191919,
               9: 0.236139169,
               11: 0.412794613,
               13: 0.621997755,
               15: 0.797306397,
               17: 0.908417508,
               19: 0.964534231,
               21: 0.992592593,
               23: 0.999775533}

def leth_fnc(speed):
  if (speed < 0): return 0
  elif (speed > 23): return 0.999775533
  else:
    return np.interp(speed, list(data_points.keys()), list(data_points.values()))

# Create the AIS raster data
# Every transmission record increments the corresponding raster pixel by
# the calculated lethality estimate of a ship/whale encounter.

for ais_file in os.listdir(ais_dir):
  if ais_file.endswith(".csv"):
    ais_filepath = os.path.join(ais_dir, ais_file)
    print(ais_filepath)
    df = pd.read_csv(ais_filepath)
    month_data = rasterio.open(tif_dir+'new/'+ais_file+'.tif', mode='w+',
                               driver=raster_data.driver, width=256, height=262,
                               count=1, dtype=raster.dtype, crs=raster_data.crs,
                               transform=raster_data.transform)
    month_read = month_data.read(1)
    df = df.reset_index()
    for index, row in df.iterrows():
      month_inds = month_data.index(row['Lon'],row['Lat'])
      month_read[month_inds[0], month_inds[1]] += (leth_fnc(row['SOG']))
    month_data.write(month_read,1)  
    month_data.close()

/content/gdrive/MyDrive/CanDev22/data/ais/AIS_2020_01_clean.csv
/content/gdrive/MyDrive/CanDev22/data/ais/AIS_2020_02_clean.csv
/content/gdrive/MyDrive/CanDev22/data/ais/AIS_2020_03_clean.csv
/content/gdrive/MyDrive/CanDev22/data/ais/AIS_2020_04_clean.csv
/content/gdrive/MyDrive/CanDev22/data/ais/AIS_2020_05_clean.csv
/content/gdrive/MyDrive/CanDev22/data/ais/AIS_2020_06_clean.csv
/content/gdrive/MyDrive/CanDev22/data/ais/AIS_2020_07_clean.csv
/content/gdrive/MyDrive/CanDev22/data/ais/AIS_2020_08_clean.csv
/content/gdrive/MyDrive/CanDev22/data/ais/AIS_2020_09_clean.csv
/content/gdrive/MyDrive/CanDev22/data/ais/AIS_2020_10_clean.csv
/content/gdrive/MyDrive/CanDev22/data/ais/AIS_2020_11_clean.csv
/content/gdrive/MyDrive/CanDev22/data/ais/AIS_2020_12_clean.csv
